<a href="https://colab.research.google.com/github/MarcosP7635/Energy/blob/main/JupyterNotebooks/nuclear_atlas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import subprocess
import pandas as pd
import numpy as np
import plotly.express as px
import plotly
running_on_colab = not "JupyterNotebooks" in subprocess.os.getcwd()
if running_on_colab:
    running_on_colab = True
    !wget https://raw.githubusercontent.com/MarcosP7635/Energy/main/ImportedData/nuclear_atlas_full.txt
    parent_dir = "/content/"
else:
    parent_dir = subprocess.os.getcwd().split("JupyterNotebooks")[0]
    seperation_character = parent_dir.split("Energy")[-1]
parent_dir

'/workspaces/Energy/'

Now to find where the data starts in the text file. From reading the paper, I know the top of the dataset reads "TABLE I".

In [20]:
file_name = "nuclear_atlas_full.txt"
if running_on_colab:
    data_dir = parent_dir
else: 
    data_dir = parent_dir + "ImportedData" + seperation_character
with open(data_dir + file_name, encoding='utf-8') as file:
    data = file.read().split("\n")
[(x, data[x]) for x in range(len(data)) 
if "TABLE I" in data[x]]

[(1057, 'TABLE I')]

In [21]:
data[1057:]

['TABLE I',
 'A. K. Jain et al.',
 'Z N AX E(keV) J\x02 T1/2 Eγ(keV) λ Decay mode Key No.',
 '4 8 12Be 2251 (1) 0+ 233.0(75) ns 144(1)',
 '2251(1)',
 'E2',
 'E0',
 '%IT = 100 2013JO06',
 '2007SH34',
 '7 9 16N 120.42 (12) 0\x03 5.25(6) μs 120.42(12) %IT = 99.99960(4)',
 '%β\x03 = 0.00040(4)',
 '1983MI10',
 '1983GA18',
 '1975PA01',
 '1967BE14',
 '9 9 18F 1121.36 (15) 5+ 162(7) ns 184 E2 %IT = 100 1995TI07',
 '9 10 19F 197.143 (4) 5/2+ 89.3(10) ns 87.3',
 '197.1 E2',
 '%IT = 100 1995TI07',
 '10 9 19Ne 238.27 (11) 5/2+ 18.0(5) ns 238.3 E2 %IT = 100 1995TI07',
 '11 11 22Na 583.11 (9) 1+ 243(2) ns 583.04(10) E2 %IT = 100 1966SU07',
 '11 13 24Na 472.2074 (8) 1+ 20.18(10) ms 472.2024(8) M3 %IT = 99.95',
 '%β\x03 ≈ 0.05',
 '1961SC09',
 '1970CH37',
 '1972BR53',
 '1980JO11',
 '13 11 24Al 425.8 (1) 1+ 130.9(13) ms 425.8(1) M3 %IT = 69.6(7)',
 '%ε+%β+ = 30.4(7)',
 '%εα = 0.028(6)',
 '2007FI14',
 '2011NI18',
 '9 17 26F 643.4 (1) 4+ 2.2(1) ms %IT ≈ 82(11)',
 '%β\x03 ≈ 18(11)',
 '2013LE03',
 '11 15 26

The rows are formatted as follows:

Z N AX E(keV) J\x02 T1/2 Eγ(keV) λ Decay mode Key No.

Thus, each row begins with an atomic number and number of neutrons, so I can just find the start of a new isomer's data by looking for two consecutive integers at the beginning of a line.

If $J^\pi$ has multiple values, then this will be the end of the line, and the next line will have the half life.

In [22]:
def is_row_start(str):
   arr = str.split(" ")
   if len(arr) > 2:
      return (arr[0].isdigit() and 
              arr[1].isdigit())
   return False
is_row_start(data[1060])

True

In [24]:
row_starts = [(r, data[r])
for r in range(len(data)) 
if is_row_start(data[r])]
row_starts

[(1060, '4 8 12Be 2251 (1) 0+ 233.0(75) ns 144(1)'),
 (1066, '7 9 16N 120.42 (12) 0\x03 5.25(6) μs 120.42(12) %IT = 99.99960(4)'),
 (1072, '9 9 18F 1121.36 (15) 5+ 162(7) ns 184 E2 %IT = 100 1995TI07'),
 (1073, '9 10 19F 197.143 (4) 5/2+ 89.3(10) ns 87.3'),
 (1076, '10 9 19Ne 238.27 (11) 5/2+ 18.0(5) ns 238.3 E2 %IT = 100 1995TI07'),
 (1077, '11 11 22Na 583.11 (9) 1+ 243(2) ns 583.04(10) E2 %IT = 100 1966SU07'),
 (1078, '11 13 24Na 472.2074 (8) 1+ 20.18(10) ms 472.2024(8) M3 %IT = 99.95'),
 (1084, '13 11 24Al 425.8 (1) 1+ 130.9(13) ms 425.8(1) M3 %IT = 69.6(7)'),
 (1089, '9 17 26F 643.4 (1) 4+ 2.2(1) ms %IT ≈ 82(11)'),
 (1092, '11 15 26Na 82.5 (5) 1+ 9(2) μs 82.5 %IT = 100 1987DUZU'),
 (1094, '13 13 26Al 228.305 (13) 0+ 6.3463(67) s %ε+%β+ = 100 2011SC22'),
 (1097, '12 19 31Mg 50.1 (2) (3/2+) 16(3) ns 50.2(2) %IT = 100 1993KL02'),
 (1099,
  '12 19 31Mg 461.0 (6) (7/2\x03) 10.5(8) ns 239.9(5) %IT = 100 2005MA96'),
 (1101, '13 19 32Al 956.6 (5) (4+) 200(20) ns 221.9(3) %IT = 100 1996RO02

In [25]:
len(row_starts)

2469

For verification, the paper says they list 2469 different isomers, and that they only include isomers with a half life > 10 ns.

Source-

free PDF I put on Zotero:
https://www.zotero.org/groups/4549380/batteries/collections/VUUF9IJG/items/BWM33GUX/attachment/665IXGIJ/reader

DOI: http://dx.doi.org/10.1016/j.nds.2015.08.001)

Assuming there are no gaps in the data, the beginning of one row must be the end of the previous row. 

In [26]:
time_units = ['d', 'h', 'min', 'ns',
's', 'y', 'μs', 'ms']
def get_data(row):
   arr = row.split(" ")
   unit_index = [i for i in range(len(arr)) 
                  if arr[i] in time_units]
   if (len(arr) > 4 and arr[0].isdigit() and arr[1].isdigit()
      and not len(unit_index) == 0):
         t_unit = arr[unit_index[0]]
         t_val = arr[unit_index[0]-1]
         isotope = arr[2]
         E_total = arr[3]
         return (isotope, E_total, t_val, t_unit)
   return "Improperly Formatted"

data_tuples = [get_data(" ".join(data[row_starts[index][0]:row_starts[index+1][0]]))
                        for index in range(len(row_starts)-1)]
data_tuples

[('12Be', '2251', '233.0(75)', 'ns'),
 ('16N', '120.42', '5.25(6)', 'μs'),
 ('18F', '1121.36', '162(7)', 'ns'),
 ('19F', '197.143', '89.3(10)', 'ns'),
 ('19Ne', '238.27', '18.0(5)', 'ns'),
 ('22Na', '583.11', '243(2)', 'ns'),
 ('24Na', '472.2074', '20.18(10)', 'ms'),
 ('24Al', '425.8', '130.9(13)', 'ms'),
 ('26F', '643.4', '2.2(1)', 'ms'),
 ('26Na', '82.5', '9(2)', 'μs'),
 ('26Al', '228.305', '6.3463(67)', 's'),
 ('31Mg', '50.1', '16(3)', 'ns'),
 ('31Mg', '461.0', '10.5(8)', 'ns'),
 ('32Al', '956.6', '200(20)', 'ns'),
 ('32Si', '5581', '27(2)', 'ns'),
 ('33Si', '1434.9', '10.2(3)', 'ns'),
 ('34Al', '0+X', '26(1)', 'ms'),
 ('34Cl', '146.36', '31.99(3)', 'min'),
 ('37K', '1380.25', '10.4(5)', 'ns'),
 ('38Cl', '671.365', '715(3)', 'ms'),
 ('38K', '130.15', '924.46(14)', 'ms'),
 ('38K', '3458.14', '21.95(11)', 'μs'),
 ('40K', '1643.639', '0.336(12)', 'μs'),
 ('42Sc', '616.28', '61.7(4)', 's'),
 ('43S', '320.9', '0.20(+14\x037)', 'μs'),
 ('43K', '738.1', '200(5)', 'ns'),
 ('43Sc', '151.4', 

In [27]:
set([x[3] for x in data_tuples if not type(x) == str])

{'d', 'h', 'min', 'ms', 'ns', 's', 'y', 'μs'}

In [28]:
bad_rows = [r for r in range(len(data_tuples)) if type(data_tuples[r]) == str]
bad_rows

[450, 469, 673]

In [29]:
[data[row_starts[bad_row][0]] for bad_row in bad_rows]

['39 62 101Y 1205.5 0.860(+90\x0380)',
 '50 52 102Sn 1969+Y (6+) 0.8(3) μs* 1996LI50',
 '49 72 121In 2448.0 (10) (21/2−) 17(2)μs # 214? 2010RE01']

Jain et al. said "the statistics are very low" for 101Y, so I will assume this means few measurements were taken and this isotope will be omitted. There are similar footnotes for 102Sn and 121In, so these two will also be omitted. 

Now to further filter the data.

In [30]:
def measured(input):
    try: 
        float(input)
        return True
    except:
        return False
measured("1000"), measured("cat")

(True, False)

These are the indices of the isotopes that have measurements for both their total decay energy and half life.

In [31]:
good_tuple_indices = [index  for index, content in enumerate(data_tuples)
                        if (measured(content[2].split("(")[0]) and 
                        measured(content[1]))]
len(good_tuple_indices)

2007

In [32]:
unit_dict = {'d':24*3600, 'h':3600, 'min':60, 'ms':.001, 'ns':10**-9, 's':1, 'y':365*24*3600, 'μs':10**-6}
isotopes = [content[0] for index, content in enumerate(data_tuples) 
            if index in good_tuple_indices]
total_energies_keV = [float(content[1]) for index, content in enumerate(data_tuples) 
            if index in good_tuple_indices]
half_lives_sec = [float(content[2].split("(")[0]) * unit_dict[content[3]] 
                for index, content in enumerate(data_tuples) 
                if index in good_tuple_indices]
data_dict = {"Isotope":isotopes, "Total Decay Energy (keV)":total_energies_keV, "Half-life (seconds)":half_lives_sec}
df = pd.DataFrame(data_dict)
df

,Isotope,Total Decay Energy (keV),Half-life (seconds)
0,12Be,2251.000,2.330000e-07
1,16N,120.420,5.250000e-06
2,18F,1121.360,1.620000e-07
3,19F,197.143,8.930000e-08
4,19Ne,238.270,1.800000e-08
...,...,...,...
2002,255Lr,38.000,2.540000e+00
2003,256Fm,1425.500,7.000000e-02
2004,257Rf,73.000,4.400000e+00
2005,257Rf,1083.200,1.060000e-04


In [ ]:
<br>Plot Subtitle</sup>

In [50]:
px.scatter(df, x = "Half-life (seconds)", y = "Total Decay Energy (keV)", 
    hover_name = "Isotope", log_x = True, log_y = True, 
    title=("Interactive Plot of Nuclear Isomers<br>" +
    "<sup>Data Retrieved from Jain et al. 2015. " + 
    "<a href=doi.org/10.1016/j.nds.2015.08.001>" + 
    "DOI:10.1016/j.nds.2015.08.001</a></sup>"))

In [34]:
[e for e in isotopes if "121Sn" in e]

['121Sn', '121Sn', '121Sn', '121Sn']